# Dependancies

pip install notebook --upgrade

pip install pip --upgrade

In [ ]:
from tensorflow.python.client import device_lib

def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]

print(get_available_devices())

In [ ]:
import platform; print(platform.platform())
import sys; print("Python", sys.version)
import numpy; print("NumPy", numpy.__version__)
import scipy; print("SciPy", scipy.__version__)
import sklearn; print("Scikit-Learn", sklearn.__version__)

In [14]:
import os
import sys
import csv

import pandas as pd
#pd.set_option('display.max_rows', None)
# pd.options.display.float_format = '{:, .2f}'.format
pd.set_option('display.max_colwidth',500)
pd.set_option('display.max_columns', 100)

import numpy as np
from numpy import save, load
from numpy import savez_compressed
from scipy.sparse import csr_matrix
from scipy.sparse import vstack
import copy
import pickle

#from scipy.misc import comb, logsumexp
from sklearn.manifold import TSNE #a tool to visualize high dimensional data
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD # dimensionality reduction using truncated SVD (AKA LSA)
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, cross_val_predict, StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn import preprocessing

import xgboost as xgb
from sklearn.model_selection import GridSearchCV

import nltk
from nltk.tokenize import word_tokenize
from nltk import FreqDist
from nltk.corpus import stopwords
from nltk.corpus import gutenberg
from nltk.collocations import *
import string #python module
import re # python regex module
from nltk.tokenize import RegexpTokenizer
from nltk.stem import SnowballStemmer
from nltk.tokenize import sent_tokenize

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

np.random.seed(0)

from sklearn.preprocessing import normalize
from functools import reduce

In [ ]:
def cosine_sim(x, y):
    try:
        if type(x) is np.ndarray: x = x.reshape(1, -1)
        if type(y) is np.ndarray: y = y.reshape(1, -1)
        d = cosine_similarity(x, y)
        d = d[0][0]
    except:
        print (x)
        print (y)
        d = 0.
    return d

# XGBoost

In [4]:
# load data

from numpy import load
target_y = load('model_target_data.npz')
target_y = target_y['arr_0']
target_y = np.ravel(target_y)
print(target_y.shape)

features_x =  load('model_data.npz')
features_x = features_x['arr_0']
print(features_x.shape)

(75385,)
(75385, 836)


In [5]:
%%time

# instantiate XGBoost classifier
boost_clf = xgb.XGBClassifier(verbosity = 2, max_depth = 6, tree_method= 'gpu_hist' ,predictor = 'gpu_predictor')

## use stratefied kfold for classification task
kfold = StratifiedKFold(n_splits=10)

# get predictions
# cross_val_predict returns an array of the same size as `y` where each entry
# is a prediction obtained by cross validation:
boost_scores = cross_val_predict(boost_clf, features_x, target_y, cv=kfold)

CPU times: user 45.2 s, sys: 13.5 s, total: 58.8 s
Wall time: 1min


In [6]:
len(boost_scores)

75385

boost_scores - 10 kfold predictions, yes?

In [9]:

## Score using scorer.py (provided in https://github.com/FakeNewsChallenge/fnc-1) on TEST set
#from scorer import score_submission, print_confusion_matrix, score_defaults, SCORE_REPORT
from score import report_score, LABELS, score_submission

In [10]:
predicted = [LABELS[int(a)] for a in boost_scores]
actual = [LABELS[int(a)] for a in target_y]


In [11]:
fold_score, _ = score_submission(actual, predicted)
max_fold_score, _ = score_submission(actual, actual)

In [17]:
score = fold_score/max_fold_score

best_score = 0
best_fold = None
    
#print("Score for fold "+ str(fold) + " was - " + str(score))
if score > best_score:
    best_score = score
    best_fold = boost_scores

In [18]:
report_score(actual,predicted)

-------------------------------------------------------------
|           |   agree   | disagree  |  discuss  | unrelated |
-------------------------------------------------------------
|   agree   |   4327    |    171    |   1030    |    53     |
-------------------------------------------------------------
| disagree  |    442    |    731    |    323    |    41     |
-------------------------------------------------------------
|  discuss  |    354    |    37     |   12845   |    137    |
-------------------------------------------------------------
| unrelated |    23     |     1     |    212    |   54658   |
-------------------------------------------------------------
Score: 32156.75 out of 34214.5	(93.98573704131289%)


93.98573704131289

In [19]:
%%time

param_grid = {
    "learning_rate": [0.1],
    'max_depth': [7],
    'min_child_weight': [9],
    'subsample': [ 0.7],
    'n_estimators': [5, 30, 100, 250, 500],
    
}

grid_clf = GridSearchCV(boost_clf, param_grid, scoring='accuracy', cv=None, n_jobs=1)
grid_clf.fit(features_x, target_y)

best_parameters = grid_clf.best_params_

print("Grid Search found the following optimal parameters: ")
for param_name in sorted(best_parameters.keys()):
    print("%s: %r" % (param_name, best_parameters[param_name]))

training_preds = grid_clf.predict(features_x)
training_accuracy = accuracy_score(target_y, training_preds)


print("")
print("Training Accuracy: {:.4}%".format(training_accuracy * 100))
print("Validation accuracy: {:.4}%".format(test_accuracy * 100))



Grid Search found the following optimal parameters: 
learning_rate: 0.1
max_depth: 7
min_child_weight: 9
n_estimators: 500
subsample: 0.7


NameError: name 'accuracy_score' is not defined

In [ ]:
boost_clf = xgb.XGBClassifier()
boost_clf.fit(X_train, y_train)

# Deep Learning Predictions

1D CNN on Headline and articleBody (at word level).  Output of CNN sent to MLP with 4 class outputs (agree, disagree, discuss, unrelated)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, MaxPooling2D, Conv2D, Activation, Dropout, GlobalAveragePooling2D
from keras import optimizers
from keras import backend


In [ ]:
nn_model = Sequential()
nn_model.add(Conv1D)